In [ ]:
import psycopg2
from configobj import ConfigObj
import sys
import os
import gdal
import json
import datetime
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import json

In [ ]:
cfg = ConfigObj("sites/bavaria.site")

with open('cfg/bands.json') as data_file:
    bandcfg = json.load(data_file)
    
conn = psycopg2.connect('postgres://{}:{}@{}/{}'.format(os.environ["POSTGIS_USER"],os.environ["POSTGIS_PASSWORD"],cfg['dbhost'],cfg['dbase']))

In [ ]:
from patchfactory import queryDates
import pylab
from pylab import imshow, show, get_cmap
import pandas as pd

def queryRaster(conn,rastertable,tiletable, tileid, date, rtype, level, bands, verbose=False):
    curs=conn.cursor()
    # convert band names to band numbers using config file

    sql="""
        select
            ST_astiff(ST_UNION(ST_CLIP(r.rast, t.geom)),ARRAY{bands})
        from
            {rastertable} r, {tiletable} t
        where
            t.id={tileid} and
            ST_INTERSECTS(r.rast,t.geom) and
            r.type='{rtype}' and
            r.level='{level}' and
            date='{date}'
        """.format(rastertable=rastertable,
        tiletable=tiletable,
        tileid=tileid,
        rtype=rtype,
        level=level,
        date=date.strftime("%Y-%m-%d"),
        bands=bands)

    if verbose: print sql
    # following https://gis.stackexchange.com/questions/130139/downloading-raster-data-into-python-from-postgis-using-psycopg2

    # Use a virtual memory file, which is named like this
    vsipath = '/vsimem/from_postgis'

    # Download raster data into Python as GeoTIFF, and make a virtual file for GDAL
    curs.execute(sql)
    gdal.FileFromMemBuffer(vsipath, bytes(curs.fetchone()[0]))

    # Read first band of raster with GDAL
    ds = gdal.Open(vsipath)
    arrays=[]
    for b in range(len(bands)):
        band = ds.GetRasterBand(b+1)
        arrays.append(band.ReadAsArray())

    # Close and clean up virtual memory file
    ds = band = None
    gdal.Unlink(vsipath)
    curs.close()

    return np.stack(arrays,axis=2)

def buff2rast(buff):
    # Use a virtual memory file, which is named like this
    vsipath = '/vsimem/from_postgis'
    
    gdal.FileFromMemBuffer(vsipath, bytes(buff))

    # Read first band of raster with GDAL
    ds = gdal.Open(vsipath)
    band=ds.GetRasterBand(1)
    
    gdal.Unlink(vsipath)
    return band.ReadAsArray()

def queryLabel(conn,tiletable, tileid, verbose=False):
    curs=conn.cursor()
    # convert band names to band numbers using config file

    
    sql="""
    select st_astiff(
    st_clip(
    st_union(
        st_asraster(st_intersection(t.geom,l.geom), 
            10::float,10::float,st_xmin(t.geom)::float, st_xmax(t.geom)::float,'8BUI',l.labelid,-9999)
    ),
    t.geom, -9999)
    )
    from {tiletable} t, fields l 
    where t.id={tileid} and ST_Intersects(t.geom, l.geom)
    group by t.geom
    
    """.format(tileid=tileid, tiletable=tiletable)
    
    if verbose: print sql
    # following https://gis.stackexchange.com/questions/130139/downloading-raster-data-into-python-from-postgis-using-psycopg2

    # Download raster data into Python as GeoTIFF, and make a virtual file for GDAL
    curs.execute(sql)
    rs = curs.fetchone()[0]
    
    arr = buff2rast(rs)
        
    # Close and clean up virtual memory file

    curs.close()

    return arr

def drawlabels(conn,tiletable,tileid):
    
    labelmap = pd.read_sql('select distinct labelid, label from labelmap where labelid is not null order by labelid', conn)
    def id2name(labelmap,labelid):
        return labelmap.loc[labelmap["labelid"]==labelid]["label"].values[0]
    
    fig, ax = plt.subplots()
    arr = queryLabel(conn,tiletable,tileid)
    im = ax.imshow(np.flipud(arr), cmap=get_cmap("Spectral"), interpolation='none')
    ax.set_title("Labels")
    uniques=np.unique(arr)
    
    cbar=fig.colorbar(im, ax=ax )
    cbar.set_ticks(uniques)
    cbar.set_ticklabels([id2name(labelmap,t) for t in uniques])
    
def plot(tileid,date,level):
    print date
    
    def m(rtype):

        def toidx(bands):
            return[bandcfg[level][rtype].index(b)+1 for b in bands]

        allbands=bandcfg[level][rtype]
        return queryRaster(conn,rastertable,tiletable, tileid, date, rtype, level, toidx(allbands)),allbands

    arr,bands = m("10m")

    fig,axs=plt.subplots(1,4,figsize=(14,10))
    i=0
    for ax in axs.reshape(-1):
        ax.imshow(arr[:,:,i],interpolation='none')
        ax.set_title(bands[i])
        i+=1

    arr,bands = m("20m")
    fig,axs=plt.subplots(2,3,figsize=(10,6))
    i=0
    for ax in axs.reshape(-1):
        ax.imshow(arr[:,:,i],interpolation='none')
        ax.set_title(bands[i])
        i+=1

    arr,bands = m("60m")
    fig,axs=plt.subplots(1,3,figsize=(9,6))
    i=0
    for ax in axs.reshape(-1):
        ax.imshow(arr[:,:,i],cmap=pylab.gray(),interpolation='none')
        ax.set_title(bands[i])
        i+=1

from ipywidgets import Layout
def drawmap(tileid):

    sql="""
    select
        ST_asgeojson(ST_Transform(t.geom,4326)), 
        ST_asgeojson(ST_Transform(st_collect(st_intersection(t.geom,l.geom)),4326))
    from {} t, fields l 
    where t.id={} and st_intersects(t.geom, l.geom)
    group by t.geom
    """.format(tiletable,tileid)
    
    cur=conn.cursor()
    cur.execute(sql)
    tile,feat = cur.fetchone()
    tilegeojson = json.loads(tile)
    featgeojson = json.loads(feat)
    cur.close()

    lon,lat = tilegeojson["coordinates"][0][0]
    center = [lat,lon]
    zoom = 15

    tilelayer = GeoJSON(data=tilegeojson)
    featlayer = GeoJSON(data=featgeojson)

    m = Map(center=center, zoom=zoom,layout=Layout(width='50%', height='300px'))

    #m.add_layer(tilelayer)
    m.add_layer(featlayer)
    
    return m

In [ ]:
dates = queryDates(conn,"bavaria")["date"].sort_values().values

level='L1C'
rastertable="bavaria"
tiletable="tiles480"

tileid=7000

drawlabels(conn, tiletable, tileid)
#datetime.datetime(2016,1,3)
plot(tileid,dates[10],level)

drawmap(tileid)

In [ ]:
level='L1C'
rastertable="bavaria"
tiletable="tiles480"
tileid=6001

drawlabels(conn,tiletable,tileid)

In [ ]:
conn.close()
cur.close()


raster ST_AsRaster(geometry geom, double precision scalex, double precision scaley, double precision gridx, double precision gridy, text pixeltype, double precision value=1, double precision nodataval=0, double precision skewx=0, double precision skewy=0, boolean touched=false);

In [ ]:


sql="""
select st_astiff(
    st_tile(
    st_clip(
    st_union(
        st_asraster(st_intersection(t.geom,l.geom), 
            10::float,10::float,st_xmin(t.geom)::float, st_xmax(t.geom)::float,'32BUI',l.code,-9999)
    ),
    t.geom, -9999),
    48,48, True
    )
    )
    from tiles480 t, osm l 
    where t.id=6604 and ST_Intersects(t.geom, l.geom)
    group by t.geom
    """


curs=conn.cursor()
curs.execute(sql)
#a = curs.fetchall()
buff = curs.fetchone()

In [ ]:
vsipath = '/vsimem/from_postgis'
gdal.FileFromMemBuffer(vsipath, bytes(buff[0]))

# Read first band of raster with GDAL
ds = gdal.Open(vsipath)

band=ds.GetRasterBand(1)
print ds.GetGeoTransform()
gdal.Unlink(vsipath)
arr = band.ReadAsArray()

In [ ]:
labelmap

In [ ]:
plt.imshow(np.flipud(arr))
plt.title(np.unique(arr))

In [ ]:
u = np.unique(arr)
arr[u[0]]=0 for unique in uniquevalues

In [ ]:
labelmap = pd.read_sql('select distinct labelid, label from labelmap where labelid is not null order by labelid', conn)

In [ ]:
labelmap

In [ ]:
labelmap.shape